# P. Visión

In [1]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from matplotlib import pyplot as plt

In [2]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [3]:
def ampliacion(imagen):    
    r1 = np.min(imagen)
    r2 = np.max(imagen)         
    imagen = np.float32(imagen)
    imagen = 255*(imagen-r1)/(r2-r1)
    imagen = np.uint8(imagen)   
    return imagen

In [4]:
path = 'C:\\Users\\alumno\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

maskHeight,maskWidth = fotosPosTrain[0].shape

path = 'C:\\Users\\alumno\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [5]:
f = np.array([[1,1,1],[1,1,1],[1,1,1]]) / 9
for i in range(len(fotosPosTrain)):
    fotosPosTrain[i] = cv2.filter2D(ampliacion(fotosPosTrain[i]),-1,f)        
for i in range(len(fotosNegTrainSinTratar)):
    fotosNegTrainSinTratar[i] = cv2.filter2D(ampliacion(fotosNegTrainSinTratar[i]),-1,f)
    

In [6]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroAleatoriasPorFoto = 3       # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]

    rowsFotoEntorno = dataNeg[0].shape[0]
    colsFotoEntorno = dataNeg[0].shape[1]

    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        for j in range(numeroAleatoriasPorFoto):
            randRow = random.randint(0,rowsFotoEntorno - rowsFotoPersona)
            randCol = random.randint(0,colsFotoEntorno - colsFotoPersona)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroAleatoriasPorFoto*i] = randomImage 
    return fotosNegTrain

In [7]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

In [8]:
def parteImagen(imagen,maskHeight,maskWidth,desp):
    arrayImagen = {}
    aux = 0
    for i in range(0,imagen.shape[0],desp):        
        for j in range(0,imagen.shape[1],desp):
            if (i+maskHeight < imagen.shape[0] and j+maskWidth < imagen.shape[1]):
                arrayImagen[aux] = imagen[i:i+maskHeight,j:j+maskWidth]
                aux = aux + 1
    return arrayImagen

In [20]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0]))
    
    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i] = hog(imagenes[i])                
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [10]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

In [21]:
# fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

ValueError: could not broadcast input array from shape (2268) into shape (6804)

In [ ]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

In [ ]:
clf = svm.SVC()
clf.fit(fotosTrain,y)

In [ ]:
clf.score(fdPosTrain,yPos)